<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# BlackHoles@Home Tutorial: Compiling the `BOINC` server on Linux

## Author: Leo Werneck

## This tutorial notebook demonstrates how to compile the `BOINC` server Linux. It focuses specifically on [Ubuntu](https://ubuntu.com), but adapting the scripts for other Linux flavors should be straightforward

## Introduction:

The [BlackHoles@Home](http://blackholesathome.net/) project allows users to volunteer CPU time so a large number of binary black holes simulations can be performed. The objective is to create a large catalog of [gravitational waveforms](https://en.wikipedia.org/wiki/Gravitational_wave), which can be used by observatories such as [LIGO](https://www.ligo.org), [VIRGO](https://www.virgo-gw.eu), and, in the future, [LISA](https://lisa.nasa.gov) in order to infer what was the source of a detected gravitational wave.

BlackHoles@Home is destined to run on the [BOINC](https://boinc.berkeley.edu) infrastructure (alongside [Einstein@Home](https://einsteinathome.org/) and [many other great projects](https://boinc.berkeley.edu/projects.php)), enabling anyone with a computer to contribute to the construction of the largest numerical relativity gravitational wave catalogs ever produced.

### Additional Reading Material:

* [BOINC's Wiki page](https://boinc.berkeley.edu/trac/wiki)
* [Debian's Wiki BOINC server guide](https://wiki.debian.org/BOINC/ServerGuide/Initialisation)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This tutorial compiles the `BOINC` server. It will also install all needed dependencies. We also provide a script to set up the server correctly.

1. [Step 1](#loading_python_nrpy_modules): Loading necessary Python/NRPy+ modules
1. [Step 2](#compilation_script): A simple script to compile the `BOINC` server    
1. [Step 3](#server_setup): Setting up the server
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='loading_python_nrpy_modules'></a>

# Step 1: Loading needed Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{loading_python_nrpy_modules}$$

We start by loading the necessary Python/NRPy+ moduels used by this tutorial notebook. We also set up the `BOINC` directory path (the default path is the current working directory).

In [1]:
# Step 1: Import necessary modules - set directories.
# Step 1.a: Import required Python modules
import sys

# Step 1.b: Add NRPy's root directory to the sys.path()
sys.path.append("..")

# Step 1.c: Load NRPy+'s command line helper module
import cmdline_helper as cmd # NRPy+: Multi-platform Python command-line interface

<a id='compilation_script'></a>

# Step 2: A simple script to compile the `BOINC` server \[Back to [top](#toc)\]
$$\label{compilation_script}$$

We will now write a simple `bash` script that will take care of downloading the `BOINC` source code and compiling the `BOINC` server.

The script performs the following tasks:

1. Install the necessary dependencies.
1. Download the [`BOINC` source code](https://github.com/BOINC/boinc) (if necessary).
1. Compile the `BOINC` server and libraries.

The following packages are required for a successfull compilation:

1. git
1. make
1. m4
1. autoconf
1. libtool
1. A C++ compiler
1. Python
1. pip
1. Python-MySQL (installed using pip)
1. mysql-server
1. apache2
1. php (with cli, gd, and mysql support)
1. pkg-config
1. Development version of libmysql++
1. Development version of libssl
1. Development version of libcurl with openssl support

In [2]:
%%writefile compile_BOINC_server.sh
#!/bin/bash
# Install all required software
sudo apt-get install -y \
  git make m4 autoconf libtool g++ python3 python-is-python3 python3-pip \
  mysql-server apache2 php php-cli php-gd php-mysql pkg-config \
  libmysql++-dev libssl-dev libcurl4-openssl-dev

# Now use pip to install MySQL for python
pip3 install mysql-client

# Download BOINC repository (if necessary)
boincdir="$(pwd)/boinc"
if [ -d "$boincdir" ]; then
  echo "BOINC directory found at $boincdir"
else
  echo "BOINC directory not found at $boincdir. Cloning from BOINC github repository..."
  git clone https://github.com/BOINC/boinc boinc
fi

# Now change directories to the BOINC directory
cd "$boincdir"

# Compile the core BOINC server and libraries
./_autosetup -f
./configure --disable-client --disable-manager
make

# Make sure the boinc_zip library is compiled
cd "${boincdir}/zip"
make

Overwriting compile_BOINC_server.sh


<a id='server_setup'></a>

# Step 3: Setting up the server \[Back to [top](#toc)\]
$$\label{server_setup}$$

We now generate a bash script which will set up the server for us. After running the `BOINC` executables that set up a new server, some file/directory permissions are still missing/misconfigured, and therefore some fixes are necessary. The script below takes care of all of the problems we have encountered when setting up a `BOINC` server so far.

The script will also add your apps to the server, if you have any. Simply change tthe `addonsdir` variable in the code below to point to the directory of your apps base directory. It assumes that the `addonsdir` directory contains at least the folders `apps` and `templates`. The `apps` directory should contain your `BOINC` applications, which should follow the `BOINC` standards (see e.g. the directory tree in [this `BOINC` tutorial](https://boinc.berkeley.edu/trac/wiki/WrapperApp)), as well as a file called `app_list.txt`.

The `app_list.txt` file should have two columns: the first one containing thte application name and the second one containing the application name in user friendly format. Lines starting with "#" are treated as comments. Here is a brief example:

```bash
# Lines starting with a # are treated as comments

# Empty lines are also OK

# The file should contain 2 columns: the first one
# with the application's (short) name and the second
# with the application's user-friendly name. Here's
# an example:

# My BOINC applications

my_app1 MyApplication1
my_app2 MyApplication2
my_app3 MyFancyApp3
```

<font color=red>NOTE:</font> the script below will request your MySQL root and your root passwords. Because of this, we will not be running it in this tutorial notebook, but you can do it by first adjusting the first few variables in the script (which set up the project and database name, as well as installation directories) and then running:

```bash
$: source setup_BOINC_server.sh
```

Once the script below finishes running without errors, you can go into the project's root directory and execute

```bash
$: sudo ./bin/start
```

to start the server. If everything went according to plan, you should then be able to access the project's website using the URL

```
http://<myip>/<projectname>
```

where `<projectname>` should be replaced with the project's name and `<myip>` by the server's IP address. If you haven't modified this on the script below, then the value of `<myip>` can be found by running

```bash
$: hostname -I | awk '{print $1}'
```

To stop the server, go to the project's root directory and execute

```bash
$: sudo ./bin/stop
```

will stop the server.

In [3]:
%%writefile setup_BOINC_server.sh
#!/bin/bash

# MySQL variable: project's database name.
dbname=blackholesathome

# MySQL variable: project's database user (I'm user my linux user here).
dbuser=bhahadm

# MySQL variable: project's database password.
dbpasswd=P@ssword1234

# BOINC variable: project name (for e.g. directories and files).
projectname=blackholesathome

# BOINC variable: project nice name (for displaying on e.g. the website).
projectnicename="BlackHoles@Home"

# BOINC variable: directory in which the project will be installed.
installroot="$(pwd)/projects"

# BOINC variable: BOINC source code directory.
boincroot="$(pwd)/boinc"

# BOINC variable: project directory.
projectroot="${installroot}/${projectname}"

# BOINC variable: this variable will be used to set your project's
#                 webpage ID. If you are running a production
#                 server, then you might want to add your static.
#                 IP address here
myip=$(hostname -I | awk '{print $1}')

# BOINC variable: project's webpage address.
hosturl="http://${myip}"

# Set this to a directory containing your BOINC applications.
# The script expects "apps" and "templates" to be subdirectories.
# If empty, then it will be unused.
addonsdir=""
appsdir=$addonsdir/apps

# Here we will pipe commands to the MySQL shell. The commands
# below set up a new database user, whose handle is specified
# by the dbuser variable set above. We then set up a database
# and grant permission to the user so that we are able to
# modify the database as we please.
printf "Please enter your MySQL password.\n"
cat <<EOMYSQL | mysql -u root -p;
DROP USER '$dbuser'@'localhost';
DROP DATABASE IF EXISTS $dbname;
CREATE USER '$dbuser'@'localhost' IDENTIFIED BY '$dbpasswd';
GRANT ALL PRIVILEGES ON $dbname.* TO '$dbuser'@'localhost';
EOMYSQL

# Now we create our project by running BOINC's make_project
# script.
# NOTE: if you are getting errors with "MySQL" module not
#       found in Python, then you might need to install
#       the MySQL Python client using sudo, i.e.:
#
#       sudo pip install mysql-client
printf "Please enter your sudo password.\n"
sudo mkdir -p "$installroot"
sudo "${boincroot}/tools/make_project" \
     --srcdir="$boincroot" \
     --url_base "$hosturl" \
     --db_name "$dbname" \
     --db_user "$dbuser" \
     --db_passwd "$dbpasswd" \
     --delete_prev_inst \
     --drop_db_first \
     --project_root "$projectroot" \
     "$projectname" "$projectnicename"

# Fix permissions of project directories. If this
# is not done, then the website is usually not
# displayed correctly. On top of that, users often
# have difficulty communicating with the server.
cd "$installroot/$projectname"
sudo chown bhahadm:bhahadm -R .
sudo chmod g+w -R .
sudo chmod 755 -R upload html/cache html/inc html/languages html/languages/compiled html/user_profile
hostname=`hostname`
sudo chgrp -R www-data log_"$hostname" upload

# Further permission fixes
echo -n "html/inc: "; sudo chmod o+x html/inc && sudo chmod -R o+r html/inc && echo "[ok]" || echo "[failed]"
echo -n "html/languages: "; sudo chmod o+x html/languages/ html/languages/compiled && echo "[ok]" || echo "[failed]"

# Copy webpage configuration to apache directory
sudo cp "${projectroot}/${projectname}.httpd.conf" /etc/apache2/sites-available
cd "${projectroot}" && sudo a2ensite "${projectname}.httpd.conf"
sudo /etc/init.d/apache2 reload

# Make sure cgi and php modules are loaded
sudo a2enmod cgi
sudo a2enmod php7.4
sudo /etc/init.d/apache2 restart

# Add the project name to the webpage file, replacing the default values.
sed -i "s/REPLACE WITH PROJECT NAME/${projectnicename}/g" "${projectroot}/html/project/project.inc"
sed -i "s/REPLACE WITH COPYRIGHT HOLDER/${projectnicename} Team/g" "${projectroot}/html/project/project.inc"

# Copy user applications apps to the project's directory
if [ "$appsdir" != "" ]; then

    # Start by copying all the apps in the application directory
    # to the project's apps directory. We also remove the "app_list.txt"
    # file, which is not used by the BOINC project.
    cp -r $appsdir/* "${projectroot}/apps/" && rm "${projectroot}/apps/app_list.txt"

    # Now we add our apps to the project by creating entries in
    # the project's project.xml file.
    # First delete the last line of the file (we'll add it back later)
    sed -i '$ d' "${projectroot}/project.xml"

    # Then add the apps to the file
    while read p; do

        # Skip if line is a comment
        [[ $p =~ ^#.* ]] && continue

        stringarray=($p)
        echo "
    <app>
        <name>${stringarray[0]}</name>
        <user_friendly_name>${stringarray[1]}</user_friendly_name>
    </app>"

    done < "${appsdir}/app_list.txt" >> "${projectroot}/project.xml"

    # Add the last line back
    echo "</boinc>" >> "${projectroot}/project.xml"

    # Copy templates to server
    cp ${addonsdir}/templates/* ${projectroot}/templates/

    # Add all the apps to the project's database
    $projectroot/bin/xadd

    # Create new app versions
    $projectroot/bin/update_versions

fi # if [ "$appsdir" != "" ]

# All done!
printf "All done!\n"

Overwriting setup_BOINC_server.sh


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-BlackHolesAtHome-Compiling_the_BOINC_server_on_Linux.pdf](Tutorial-BlackHolesAtHome-Compiling_the_BOINC_server_on_Linux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [4]:
!cp ../latex_nrpy_style.tplx .
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-BlackHolesAtHome-Compiling_the_BOINC_server_on_Linux")
!rm -f latex_nrpy_style.tplx

Created Tutorial-BlackHolesAtHome-Compiling_the_BOINC_server_on_Linux.tex,
    and compiled LaTeX file to PDF file Tutorial-BlackHolesAtHome-
    Compiling_the_BOINC_server_on_Linux.pdf
